# Hands-on 1: Ingestion and Chunking

## Problem

DEFINITION:

https://docs.llamaindex.ai/en/stable/examples/retrievers/relative_score_dist_fusion/

## Code

In [26]:
# if running on colab uncomment the those lines
# %pip install llama-index>=0.12.2
# %pip install llama-index-retrievers-bm25>=0.50
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [27]:
# load .env
from dotenv import load_dotenv
load_dotenv()

# import os
# os.environ["OPENAI_API_KEY"] = "sk-proj-..."

True

In [28]:
from llama_index.core import SimpleDirectoryReader
import nest_asyncio

nest_asyncio.apply()

documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [29]:

from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from rich import print
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.memory import ChatMemoryBuffer

In [31]:
splitter = SentenceSplitter(chunk_size=256)

index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], show_progress=True
)

vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=3
)


Generating embeddings: 100%|██████████| 587/587 [00:10<00:00, 57.84it/s]


In [32]:
QUERY_GEN_PROMPT="""You are a helpful assistant that generates multiple search queries based on a single input query. Generate {num_queries} search queries, one on each line, related to the following input query:
Query: {query}
Queries:"""

In [33]:
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=10,
    num_queries=3,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT,
)

In [34]:
CHAT_PROMPT = """You're a chatbot, able to have normal conversations, about an essay discussing Paul Grahams life. 
Here are the relevant documents for the context:
{context_str}
Instruction: use the previous chat history, or the context aboute, to interact with the user."""

In [49]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.chat_engine.condense_plus_context import CondensePlusContextChatEngine
memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
query_engine = RetrieverQueryEngine(retriever)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    query_engine=query_engine,
    memory=memory,
    context_prompt=CHAT_PROMPT,
)

In [ ]:
# from llama_index.core.query_engine import RetrieverQueryEngine
# from llama_index.legacy.chat_engine.condense_question import CondenseQuestionChatEngine

# memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
# query_engine = RetrieverQueryEngine(retriever)

# chat_engine = CondenseQuestionChatEngine.from_defaults(
#     query_engine=query_engine,
#     memory=memory,
#     context_prompt=CHAT_PROMPT,

# )

In [50]:
chat_engine.chat("What did Paul Graham do in the summer of 1995?")

Generated queries:
1. What projects did Paul Graham work on in the summer of 1995?
2. Did Paul Graham write any influential essays or articles in the summer of 1995?


AgentChatResponse(response='In the summer of 1995, Paul Graham organized something called the Summer Founders Program. He and his team decided to fund a bunch of startups at once by inviting undergrads to start their own startups instead of taking temporary jobs at tech companies. They used a building in Cambridge as their headquarters, had weekly dinners, and brought in experts on startups to give talks. This program attracted a lot of interest, with 225 applications received, including from recent graduates. It was the beginning of a more serious endeavor than they had initially intended.', sources=[ToolOutput(content='file_path: c:\\Users\\n.fretti\\Desktop\\projects\\rag_and_roll\\lesson_02\\data\\paul_graham\\paul_graham_essay.txt\n\nSo while we probably wouldn\'t make much money out of it, we\'d at least get to practice being investors on them, and they for their part would probably have a more interesting summer than they would working at Microsoft.\n\nWe\'d use the building I o

In [51]:
chat_engine.chat("Why did he organize it?")

Generated queries:
1. What were the goals of the Summer Founders Program organized by Paul Graham in 1995?
2. How did the Summer Founders Program impact the startup ecosystem in 1995 and beyond?


AgentChatResponse(response='Paul Graham organized the Summer Founders Program as a way to gain experience as investors. They wanted to fund multiple startups at once to practice being investors and to provide a more interesting summer experience for undergrads who would otherwise be working at companies like Microsoft. The program allowed them to scale startup funding and provide support to a larger number of startups simultaneously. Additionally, it helped address the issue of isolation that many founders face by creating a community of like-minded individuals who could share their experiences and solutions to common problems.', sources=[ToolOutput(content='file_path: c:\\Users\\n.fretti\\Desktop\\projects\\rag_and_roll\\lesson_02\\data\\paul_graham\\paul_graham_essay.txt\n\nWe knew undergrads were deciding then about summer jobs, so in a matter of days we cooked up something we called the Summer Founders Program, and I posted an announcement on my site, inviting undergrads to apply. 

In [52]:
chat_engine.reset()

In [48]:
chat_engine.chat("What did I ask you?")

Generated queries:
1. How can I improve my memory recall during conversations?
2. Techniques for effective communication and active listening skills.


AgentChatResponse(response='You asked me about the content and details provided in the context information.', sources=[ToolOutput(content='You asked me about the content and details provided in the context information.', tool_name='query_engine', raw_input={'query': 'What did I ask you in our conversation earlier?'}, raw_output=Response(response='You asked me about the content and details provided in the context information.', source_nodes=[NodeWithScore(node=TextNode(id_='93e9cadb-52a0-4526-a3ab-708144e647be', embedding=None, metadata={'file_path': 'c:\\Users\\n.fretti\\Desktop\\projects\\rag_and_roll\\lesson_02\\data\\paul_graham\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-12-04', 'last_modified_date': '2024-12-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type